# mount google drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

# set working directory

In [10]:
import os
os.chdir("/home/michael/gitrepos/CC0325/codes/matvec/")

# matvec in (pure) python

In [ ]:
import pandas as pd
import numpy as np
from time import time
from tqdm import tqdm

def matvec(matrix, vector):
    result = np.zeros(vector.shape)
    for i in range(matrix.shape[0]):
        for j in range(vector.shape[0]):
            result[i] += matrix[i, j] * vector[j]
    return result

def profile_matvec(NROWS):
    times = []
    for nrows in tqdm(NROWS):
        vector = np.random.rand(nrows, 1)
        matrix = np.random.rand(nrows, nrows)
        start = time()
        y = matvec(matrix, vector)
        end = time()
        times.append((end - start) * 1e6)  # Convert to microseconds
    return times

NROWS = [100, 200, 300, 500, 800, 1000, 1250, 1500, 1750, 2000]
times = profile_matvec(NROWS)

df_py = pd.DataFrame({'NROWS': NROWS, 'Time': times})
df_py

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import sympy as sp

def plot_polynomial_fit(df):
    # Fit a polynomial to the data
    coefficients = np.polyfit(df['NROWS'], df['Time'], 2)
    poly = np.poly1d(coefficients)

    # Show the polynomial fit equation
    x = sp.Symbol('x')
    poly_equation = sp.Poly(poly(x), x)
    print(f"Polynomial Fit Equation: {poly_equation}")

    # Plot the data points
    plt.plot(df['NROWS'], df['Time'], marker='o', label='Data Points')

    # Plot the polynomial fit, dashed line
    plt.plot(df['NROWS'], poly(df['NROWS']), label='Poly Fit', linestyle='--')

    plt.legend()
    plt.show()

plot_polynomial_fit(df_py)


# matvec using C++

In [ ]:
!make clean
!make run

In [ ]:
# read the output file
df_cpp = pd.read_csv('output.txt')
df_cpp.head()

plot_polynomial_fit(df_cpp)


In [ ]:
df_merged = pd.merge(df_py, df_cpp, on='NROWS', suffixes=['_py', '_cpp'])
df_merged['speedup'] = df_merged['Time_py'] / df_merged['Time_cpp']
df_merged

# matvec using numpy

In [ ]:
def matvec_numpy(NROWS):
    times = []
    for nrows in tqdm(NROWS):
        vector = np.random.rand(nrows, 1)
        matrix = np.random.rand(nrows, nrows)
        start = time()
        y = matrix @ vector
        end = time()
        times.append((end - start) * 1e6)  # Convert to microseconds
    return times

NROWS = df_cpp['NROWS']
times = matvec_numpy(NROWS)
df_np = pd.DataFrame({'NROWS': NROWS, 'Time': times})
df_np


In [ ]:
plot_polynomial_fit(df_np)

In [ ]:
df_merged_np = pd.merge(df_np, df_cpp, on='NROWS', suffixes=['_np', '_cpp'])
df_merged_np['speedup'] = df_merged_np['Time_np'] / df_merged_np['Time_cpp']
df_merged_np


# Numpy vs C++

NumPy foi mais rápido em comparação ao nosso C++, pois ele realiza operações em arrays usando código C/Fortran otimizado (BLAS), o que evita os loops no Python e aumenta consideravelmente a velocidade.

**Referências**
1. [https://numpy.org/](https://numpy.org/)
1. [https://en.wikipedia.org/wiki/NumPy](https://en.wikipedia.org/wiki/NumPy)
1. [https://www.intel.com/content/www/us/en/docs/onemkl/developer-reference-c/2024-2/cblas-gemv.html](https://www.intel.com/content/www/us/en/docs/onemkl/developer-reference-c/2024-2/cblas-gemv.html)
1. [https://superfastpython.com/what-is-blas-and-lapack-in-numpy/](https://superfastpython.com/what-is-blas-and-lapack-in-numpy/)
1. [https://www.tomasbeuzen.com/python-programming-for-data-science/chapters/chapter6-numpy-addendum.html](https://www.tomasbeuzen.com/python-programming-for-data-science/chapters/chapter6-numpy-addendum.html)


# Exercício
1. Na função "profile_matvec" do arquivo "matvec.cpp", substitua a chamada da função "matvec" por "matvec_blas", refaça os experimentos e comente os resultados. A função "matvec_blas" utiliza a função "cblas_dgemv" da biblioteca BLAS.

In [ ]:
!make clean
!make run


In [ ]:
df_blas = pd.read_csv('output.txt')
df_blas.head()

plot_polynomial_fit(df_blas)